In [1]:
import pandas as pd
import re
import os
import numpy as np
import glob
from datetime import datetime
import geopandas as gpd
import json
from shapely.geometry import shape, GeometryCollection
from rasterio.plot import show
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib import colors
from utils import FeatureExtraction, feature_matching, decimal_coords
from exif import Image as exifimg
from PIL import Image

import shutil
import rasterio
import cv2
%matplotlib inline

In [2]:
# 가능한 font list 확인
import matplotlib.font_manager as fm
f = [f.name for f in fm.fontManager.ttflist]
# print(f)

plt.rc('font', family='Malgun Gothic')

In [3]:
import boto3
from decimal import Decimal
# Get the service resource.
dynamodb = boto3.resource('dynamodb', region_name='ap-northeast-2')
table = dynamodb.Table('dsnia_imagelabelsV2')
# table = dynamodb.Table('dsnia_imagelabels')

In [4]:
from boto3.dynamodb.conditions import Key, Attr
from botocore.exceptions import ClientError

In [5]:

courseid = "MGC001"
date = "20230816"


response = table.query(IndexName='courseid-id-index', KeyConditionExpression=Key('courseid').eq(courseid),  FilterExpression=Attr('date').eq(date))
data =  response['Items']

while 'LastEvaluatedKey' in response:
    response = table.query(IndexName='courseid-id-index', KeyConditionExpression=Key('courseid').eq(courseid), 
     ExclusiveStartKey=response['LastEvaluatedKey'] ,FilterExpression=Attr('date').eq(date) )    
    # response_ = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
    data.extend(response['Items'])

In [6]:
len(data)

9184

In [7]:
folderList = list(set([x['json'].get('destFolder') for x in data]))
# folderList

{
    "path": "20230823/9_레이크4H/패취류,섬머패취,켄터키,티",
    "file_cnt": 478,
    "labeled_file_cnt": 0
  }

In [8]:
folder_ = folderList[0]

folderInfo = []

for folder_ in folderList:
  res = {}
  res['path'] = folder_
  res['file_cnt'] = len([x for x in data if x['json'].get('destFolder') == folder_])
  res['labeled_file_cnt'] = len([x for x in data if x['json'].get('destFolder') == folder_ and len(x['json'].get('label')) !=0 ])
  res['TBD_file_cnt'] = len([x for x in data if x['json'].get('destFolder') == folder_ and len([label_ for label_ in x['json'].get('label') if label_.get('level1') == "TBD"])])
  print(res)
  folderInfo.append(res)

{'path': '20230816/5.동해3H/미기재_1', 'file_cnt': 402, 'labeled_file_cnt': 402, 'TBD_file_cnt': 0}
{'path': '20230816/6.태백1H/미기재_3', 'file_cnt': 52, 'labeled_file_cnt': 52, 'TBD_file_cnt': 0}
{'path': '20230816/2.태백5H/미기재_3', 'file_cnt': 66, 'labeled_file_cnt': 0, 'TBD_file_cnt': 0}
{'path': '20230816/2.태백5H/미기재_1', 'file_cnt': 501, 'labeled_file_cnt': 501, 'TBD_file_cnt': 0}
{'path': '20230816/2.태백5H/미기재_2', 'file_cnt': 501, 'labeled_file_cnt': 501, 'TBD_file_cnt': 0}
{'path': '20230816/1.태백3H/미기재_3', 'file_cnt': 501, 'labeled_file_cnt': 480, 'TBD_file_cnt': 0}
{'path': '20230816/6.태백1H/미기재_1', 'file_cnt': 501, 'labeled_file_cnt': 501, 'TBD_file_cnt': 0}
{'path': '20230816/6.태백1H/미기재_0', 'file_cnt': 501, 'labeled_file_cnt': 501, 'TBD_file_cnt': 0}
{'path': '20230816/6.태백1H/미기재_2', 'file_cnt': 501, 'labeled_file_cnt': 501, 'TBD_file_cnt': 0}
{'path': '20230816/1.태백3H/미기재_2', 'file_cnt': 501, 'labeled_file_cnt': 501, 'TBD_file_cnt': 0}
{'path': '20230816/3.동해1H/미기재_3', 'file_cnt': 501, 'lab

In [9]:
folderInfo

[{'path': '20230816/5.동해3H/미기재_1',
  'file_cnt': 402,
  'labeled_file_cnt': 402,
  'TBD_file_cnt': 0},
 {'path': '20230816/6.태백1H/미기재_3',
  'file_cnt': 52,
  'labeled_file_cnt': 52,
  'TBD_file_cnt': 0},
 {'path': '20230816/2.태백5H/미기재_3',
  'file_cnt': 66,
  'labeled_file_cnt': 0,
  'TBD_file_cnt': 0},
 {'path': '20230816/2.태백5H/미기재_1',
  'file_cnt': 501,
  'labeled_file_cnt': 501,
  'TBD_file_cnt': 0},
 {'path': '20230816/2.태백5H/미기재_2',
  'file_cnt': 501,
  'labeled_file_cnt': 501,
  'TBD_file_cnt': 0},
 {'path': '20230816/1.태백3H/미기재_3',
  'file_cnt': 501,
  'labeled_file_cnt': 480,
  'TBD_file_cnt': 0},
 {'path': '20230816/6.태백1H/미기재_1',
  'file_cnt': 501,
  'labeled_file_cnt': 501,
  'TBD_file_cnt': 0},
 {'path': '20230816/6.태백1H/미기재_0',
  'file_cnt': 501,
  'labeled_file_cnt': 501,
  'TBD_file_cnt': 0},
 {'path': '20230816/6.태백1H/미기재_2',
  'file_cnt': 501,
  'labeled_file_cnt': 501,
  'TBD_file_cnt': 0},
 {'path': '20230816/1.태백3H/미기재_2',
  'file_cnt': 501,
  'labeled_file_cnt': 50

In [11]:
AWS_folder = os.getcwd()

b_folder = os.path.join(AWS_folder,'BACKUPS')
b_folder

src_folder = os.path.join(AWS_folder,'dataDownloaded')
src_folder

out_folder = os.path.join(AWS_folder,'filepathBackUp')
out_folder

'c:\\Users\\yc463\\Documents\\JamesLocal\\DS\\Drone\\dsmapv2\\src\\AWS\\filepathBackUp'

In [13]:
save_name = os.path.join(out_folder,'filepath_'+courseid + '_'+date+'.json')
with open(save_name, "w", encoding='utf-8') as final:
   json.dump(folderInfo, final , ensure_ascii=False)